## Estado de Capitales Mínimos

Se genera un dataset que permita analizar de la resolución RESOL-2018-1116-APN-SSN#MHA el punto 23.
En particular:  

**A. EMISION < 5% en una rama** 
Las aseguradoras deberán acreditar al cierre de cada ejercicio anual una emisión total superior al CINCO POR CIENTO (5%) del capital mínimo exigido en dicho inciso.
    * Patrimoniales: 90.000.000 
    * Vida: 36.000.000

A efectos de realizar el cálculo del párrafo precedente, se entenderá por emisión total a la sumatoria de la emisión de una o más ramas en las que se haya autorizado a operar a la entidad y que se encuentren incluidas en el inciso i) del punto 30.1.1.1.

In [1]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math
import time
import datetime

%matplotlib inline

#pd.options.display.max_rows = 10
pd.options.display.float_format = '{:,.5f}'.format
plt.rcParams['figure.figsize'] = (7, 5)

Ramos y subramos

In [3]:
sub_ramos=pd.read_excel(r'C:\Users\eangenello\Desktop\PYTHON\input\ECM\Ramos - Subramo y Tipo.xlsx', sheet_name='tabla ramos')

Ramos Autorizados por Cias

In [4]:
autorizaciones=pd.read_excel(r'C:\Users\eangenello\Desktop\PYTHON\input\ECM\Autorizaciones.xlsx', sheet_name='Ramos aut x cia', usecols='A,C,E')

Primas

In [5]:
PE=pd.read_excel(r'C:\Users\eangenello\Desktop\PYTHON\input\ECM\ANEXO_12.xlsx', usecols='A, C:E')
PE=PE.rename(columns = {'año':'periodo'})
PE['año'] = pd.DatetimeIndex(PE['periodo']).year
PE['trimestre'] = pd.DatetimeIndex(PE['periodo']).month

#Selecciono el período de análisis
PE=PE[(PE['año']==2019)&(PE['trimestre']==2)]

#Reordeno columnas
PE = PE[['cod_cia', 'periodo', 'año', 'trimestre', 'cod_subramo', 'Primas_Emitidas']]

Compañías que surgen del dataset de ECM del SINENSUP

In [101]:
dt_cias=pd.read_excel(r'C:\Users\eangenello\Desktop\PYTHON\input\ECM\Cias ECM.xlsx')
dt_cias=dt_cias.rename(columns = {'Cia ID':'cod_cia'})

Tipos de Ramos

In [50]:
ramos=pd.read_excel(r'C:\Users\eangenello\Desktop\PYTHON\input\ECM\Ramos - Subramo y Tipo.xlsx', sheet_name='tipo ramos')

Construcción del dataset

In [113]:
# Me quedo con una parte del sataset de ramos

sub_ramos_aut = sub_ramos.drop({'cod_subramo','des_subramo'},1)
sub_ramos_aut = sub_ramos_aut.drop_duplicates()
sub_ramos_aut = sub_ramos_aut[(sub_ramos_aut['cod_ramo']!='3.000.99')&(sub_ramos_aut['cod_ramo']!='9999')]

sub_ramos_aut.head()

,cod_ramo,des_ramo,Tipo
0,1.010.00,Incendio,Patrimonial
1,1.020.00,Combinado Familiar e Integral,Patrimonial
4,1.030.00,Automotores,Patrimonial
7,1.040.00,Transporte Público de Pasajeros,Patrimonial
8,1.050.00,Riesgos del Trabajo,ART


In [114]:
# Ramo un dataset con cod cia y todos los ramos

cias = autorizaciones['cod_cia'].unique()
ramos_cias = pd.DataFrame()

for i in cias:
    dataset = sub_ramos_aut.copy()
    dataset['cod_cia'] = i
    ramos_cias = ramos_cias.append(dataset)

In [115]:
autorizaciones.head()

,cod_cia,cod_ramo,Autorizacion
0,2,1.080.00,Autorizada
1,2,1.100.00,Autorizada
2,2,2.030.00,Autorizada
3,2,1.030.00,Autorizada
4,2,1.170.00,Autorizada


In [116]:
# Agrego la descripción y tipo de ramo al dataset de autorizaciones

autor_tipo_ramo = ramos_cias.merge(autorizaciones, how='left', on=['cod_ramo', 'cod_cia'])

# Reemplazo los nulos por 'Sin Autorización'
autor_tipo_ramo['Autorizacion'] = autor_tipo_ramo['Autorizacion'].fillna('Sin_Autorización')
autor_tipo_ramo.head()

,cod_ramo,des_ramo,Tipo,cod_cia,Autorizacion
0,1.010.00,Incendio,Patrimonial,2,Autorizada
1,1.020.00,Combinado Familiar e Integral,Patrimonial,2,Autorizada
2,1.030.00,Automotores,Patrimonial,2,Autorizada
3,1.040.00,Transporte Público de Pasajeros,Patrimonial,2,Sin_Autorización
4,1.050.00,Riesgos del Trabajo,ART,2,Sin_Autorización


In [117]:
# Agrego a prima emitida el ramo
PE_ramo = PE.merge(sub_ramos, how='left', on=['cod_subramo'])

# Agrupo las primas emitidas por ramo
PE_ramo = PE_ramo.groupby(['cod_cia','cod_ramo', 'des_ramo'], as_index=False)['Primas_Emitidas'].sum()
PE_ramo.head()

,cod_cia,cod_ramo,des_ramo,Primas_Emitidas
0,2,1.010.00,Incendio,754062663
1,2,1.020.00,Combinado Familiar e Integral,303287161
2,2,1.030.00,Automotores,1533538660
3,2,1.080.00,Responsabilidad Civil,631732086
4,2,1.090.00,Robo y Riesgos Similares,13823931


In [118]:
# JUNTO DATASETS: Autorizaciones con primas 

ECM = autor_tipo_ramo.merge(PE_ramo, how='left', on=['cod_cia', 'cod_ramo', 'des_ramo'] )

# Reemplazo los vales nulos
ECM = ECM.replace(to_replace=np.nan, value=0)

In [119]:
ECM.head(8)

,cod_ramo,des_ramo,Tipo,cod_cia,Autorizacion,Primas_Emitidas
0,1.010.00,Incendio,Patrimonial,2,Autorizada,"754,062,663.00000"
1,1.020.00,Combinado Familiar e Integral,Patrimonial,2,Autorizada,"303,287,161.00000"
2,1.030.00,Automotores,Patrimonial,2,Autorizada,"1,533,538,660.00000"
3,1.040.00,Transporte Público de Pasajeros,Patrimonial,2,Sin_Autorización,0.00000
4,1.050.00,Riesgos del Trabajo,ART,2,Sin_Autorización,0.00000
5,1.070.00,Riesgos Agropecuarios y Forestales,Patrimonial,2,Autorizada,0.00000
6,1.080.00,Responsabilidad Civil,Patrimonial,2,Autorizada,"631,732,086.00000"
7,1.090.00,Robo y Riesgos Similares,Patrimonial,2,Autorizada,"13,823,931.00000"


In [120]:
# Calculo el porcentaje de prima emitida sobre el capital por ramo

ECM['Comprobacion'] = np.where(ECM['Tipo'] == 'Patrimonial', ECM['Primas_Emitidas']/90000000 , ECM['Primas_Emitidas']/36000000)

In [121]:
# Agrego Denominación corta

ECM=ECM.merge(dt_cias, how='left', on=['cod_cia'])

In [122]:
# Cambio el orden de las columnas

ECM = ECM[['cod_cia','DenominacionCorta','cod_ramo','des_ramo','Tipo','Autorizacion','Primas_Emitidas','Comprobacion']]

In [123]:
ECM.head()

,cod_cia,DenominacionCorta,cod_ramo,des_ramo,Tipo,Autorizacion,Primas_Emitidas,Comprobacion
0,2,SMG,1.010.00,Incendio,Patrimonial,Autorizada,"754,062,663.00000",8.37847
1,2,SMG,1.020.00,Combinado Familiar e Integral,Patrimonial,Autorizada,"303,287,161.00000",3.36986
2,2,SMG,1.030.00,Automotores,Patrimonial,Autorizada,"1,533,538,660.00000",17.03932
3,2,SMG,1.040.00,Transporte Público de Pasajeros,Patrimonial,Sin_Autorización,0.00000,0.00000
4,2,SMG,1.050.00,Riesgos del Trabajo,ART,Sin_Autorización,0.00000,0.00000


In [125]:
ECM.to_excel(r'C:\Users\eangenello\Desktop\PYTHON\output\ECM\Dataset_ECM_Resol1116.xlsx', sheet_name='Dataset_ECM', index=False)

In [90]:
# Calculo el porcentaje de prima emitida sobre el capital por ramo

#ECM['Comprobacion']=1

#for i in ECM.index:
#    if ECM.Comprobacion[i]=='Patrimonial':
#        ECM.Comprobacion[i]=ECM.Primas_Emitidas[i]/90000000
#    else:
#        ECM.Comprobacion[i]=ECM.Primas_Emitidas[i]/36000000